In [7]:
import time
from __future__ import division # safety with double division
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np
import random


In [8]:
random.seed(1024)
Opt = SolverFactory("gurobi")

# Clustering for Centroids
In this linear programming optimization, it will be optimizing for the Centroids location relative to the fixed assignments;

### M1:
This LP will fix the assignment of the Centroids and optimize for the Lowest possible distance of $C_i$ 

In [9]:
M1 = AbstractModel()
M1.name = "Clustering Centroid-Distance LP"

### M2
This LP will fix the Centroid Locations and redo the assignement. 

In [10]:
M2 = AbstractModel()
M2.name = "Clustering Assignment-LP"

## Parameters
Both LP's have same starting parameter that characterize the problem.
- **d**: number of dimensions
- **n**: number of points to cluster
- **k**: number of clusters to generate

In [11]:
M1.NumberOfDimensions = Param(within=NonNegativeIntegers)
M1.NumberOfPoints = Param(within=NonNegativeIntegers)
M1.NumberOfClusters = Param(within=NonNegativeIntegers)

In [12]:
M2.NumberOfDimensions = Param(within=NonNegativeIntegers)
M2.NumberOfPoints = Param(within=NonNegativeIntegers)
M2.NumberOfClusters = Param(within=NonNegativeIntegers)

## Set
- **Dimension Index (D)**: Set consisting of all possible possible dimensions an arbitrary point i.e. $\{x_1, x_2, x_3 ... x_d\}$
- **Points (P)**: Set consisting of all indexes for Points in the system. $\{p_1, p_2, p_3 ... p_n\}$
- **Cluster Index (C)**: Set consisting of possible ClusterIndex. $\{c_1, c_2, c_3 ... c_k\}$

In [13]:
M1.DimensionIndex = RangeSet(1,M1.NumberOfDimensions)
M1.PointsIndex = RangeSet(1,M1.NumberOfPoints)
M1.ClusterIndex = RangeSet(1,M1.NumberOfClusters)

In [14]:
M2.DimensionIndex = RangeSet(1,M2.NumberOfDimensions)
M2.PointsIndex = RangeSet(1,M2.NumberOfPoints)
M2.ClusterIndex = RangeSet(1,M2.NumberOfClusters)

## Inputs
- **Point**: $P_{i,d}$ where $i$ $\in$ PointsIndex and $j$ $\in$ DimensionIndex 

In [15]:
M1.Point = Param(M1.PointsIndex,M1.DimensionIndex, within=Reals)

In [16]:
M2.Point = Param(M2.PointsIndex,M2.DimensionIndex, within=Reals)

## Possible Variables
- **Centroid**: $C_{i,d}$ where i $\in$ ClusterIndex and d $\in$ dimensionalIndex 
- **Assignment**: $A_{i,j}$ where i $\in$ pointsIndex and j $\in$ clusteringIndex
- **Slack**: $S_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D
- **Z**: The Upper Bound for the slack variables

### M1 Variables
- **Centroid**: $C_{i,d}$ where i $\in$ ClusterIndex and d $\in$ D
- **Slack**: $S_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D
- **Z**: The Upper Bound for the slack variables

In [17]:
M1.Centroid=Var(M1.ClusterIndex, M1.DimensionIndex, within=Reals)
M1.Slack = Var(M1.PointsIndex,M1.ClusterIndex,M1.DimensionIndex, within=Reals)
M1.Z = Var(M1.PointsIndex,M1.ClusterIndex, within=NonNegativeReals)

### M2 Variables
- **Assignment**: $A_{i,j}$ where i $\in$ pointsIndex and j $\in$ clusteringIndex
- **Slack**: $S_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D
- **Z**: The Upper Bound for the slack variables

In [18]:
M2.Assignment = Var(M2.PointsIndex, M2.ClusterIndex, within=Binary)
M2.Slack = Var(M2.PointsIndex,M2.ClusterIndex,M2.DimensionIndex, within=Reals)
M2.Z = Var(M2.PointsIndex,M2.ClusterIndex, within=NonNegativeReals)

### M1 Fixed Value

In [19]:
M1.Assignment=Param(M1.PointsIndex, M1.ClusterIndex, default=0,within=Binary, mutable=True)

### M2 Fixed Value

In [20]:
M2.Centroid=Param(M2.ClusterIndex, M2.DimensionIndex,default=0,within=Reals, mutable=True)

# Model M1
## Objective Function for M1
$$ \textbf{min}  \;\sum_{i\in P}\sum_{j\in C} Z_{i,j} $$

In [21]:
def ObjectiveFunction(M):
    return sum(\
               M.Z[i,j]\
               for i in M.PointsIndex \
               for j in M.ClusterIndex)
M1.Distance = Objective(rule=ObjectiveFunction, sense=minimize)
    

### Constraint 1: Distance Constraint
Used to convert distance metric into 1-norm
$$0=A_{i,j}\cdot(P_{i,d}-C_{j,d})+(S_{i,j,d}) \qquad \forall i \in P, j\in C, d \in D $$

In [22]:
def DistanceConstraint(M,i,j,d):
    return 0 == M.Assignment[i,j]*(M.Point[i,d]-M.Centroid[j,d])+M.Slack[i,j,d]
M1.Norm = Constraint(M1.PointsIndex, M1.ClusterIndex, M1.DimensionIndex, rule = DistanceConstraint)

### Constraint 2: Within Lower Bound
$$ -Z_{i,j} \leq S_{i,j,d} \qquad \forall i \in P, j\in C, d \in D $$

In [23]:
def LowerBoundConstraint1(M,i,j,d):
    return -M.Z[i,j] <=M.Slack[i,j,d]
M1.LowerZBound = Constraint(M1.PointsIndex, M1.ClusterIndex, M1.DimensionIndex, rule = LowerBoundConstraint1)

### Constraint 3: With Upper Bound
$$ Z_{i,j} \geq S_{i,j,d} \qquad \forall i \in P, j\in C, d \in D $$

In [24]:
def UpperBoundConstraint1(M,i,j,d):
    return M.Z[i,j] >=M.Slack[i,j,d]
M1.UpperZBound = Constraint(M1.PointsIndex, M1.ClusterIndex, M1.DimensionIndex, rule = UpperBoundConstraint1)

# Model M2
## Objective Function for M2
$$ \textbf{min}  \;\sum_{i\in P}\sum_{j\in C} Z_{i,j} $$

In [25]:
def ObjectiveFunction(M):
       return sum(\
               M.Z[i,j]\
               for i in M.PointsIndex \
               for j in M.ClusterIndex)
M2.Distance = Objective(rule=ObjectiveFunction, sense=minimize)

### Constraint 1: Distance Constraint
Used to convert distance metric into 1-norm
$$0=A_{i,j}\cdot(P_{i,d}-C_{j,d})+(S_{i,j,d}) \qquad \forall i \in P, j\in C, d \in D $$

In [26]:
def DistanceConstraint(M,i,j,d):
    return 0 == M.Assignment[i,j]*(M.Point[i,d]-M.Centroid[j,d])+M.Slack[i,j,d]
M2.Norm = Constraint(M2.PointsIndex, M2.ClusterIndex, M2.DimensionIndex, rule = DistanceConstraint)

### Constraint 2: Within Lower Bound
$$ -Z_{i,j} \leq S_{i,j,d} \qquad \forall i \in P, j\in C, d \in D $$

In [27]:
def LowerBoundConstraint2(M,i,j,d):
    return -M.Z[i,j]<=M.Slack[i,j,d]
M2.LowerZBound = Constraint(M2.PointsIndex, M2.ClusterIndex, M2.DimensionIndex, rule = LowerBoundConstraint2)

### Constraint 3: With Upper Bound
$$ Z_{i,j} \geq S_{i,j,d} \qquad \forall i \in P, j\in C, d \in D $$

In [28]:
def UpperBoundConstraint2(M,i,j,d):
    return M.Z[i,j] >=M.Slack[i,j,d]
M2.UpperZBound = Constraint(M2.PointsIndex, M2.ClusterIndex, M2.DimensionIndex, rule = UpperBoundConstraint2)

### Constraint 4: Non-Empty Cluster
$$ 1\leq \sum_{i \in P} A_{i,j} \qquad \forall j \in C $$

In [29]:
def NonEmptyCluster(M, j):
    return 1<=sum(M.Assignment[i,j] for i in M.PointsIndex)
M2.NonEmptyBalance = Constraint(M2.ClusterIndex, rule = NonEmptyCluster)

### Constraint 5: Singular Assignment 
$$ 1 = \sum_j A_{i,j} \qquad \forall i \in P$$

In [30]:
def SingularAssignment(M, i):
    return 1==sum(M.Assignment[i,j] for j in M.ClusterIndex)
M2.SingularAssignmentBalance = Constraint(M2.PointsIndex, rule = SingularAssignment)

## Create Problem and Solver Instance

In [31]:
folder  = "../Data/Mushroom/"
dat_location ="mushroom_lp"
dat_file = folder+dat_location+".dat"

In [33]:
start_time = time.time()
original_instance2 = M2.create_instance(dat_file)
clusters = original_instance2.NumberOfClusters.value
points = original_instance2.NumberOfPoints.value
dimensions = original_instance2.NumberOfDimensions.value

In [34]:
cluster_column = ["iteration", "index"]+["x_"+str(d) for d in range(1,dimensions+1)]
centroid_DF = pd.DataFrame(columns=cluster_column)

In [35]:
assignment_column  = ["iteration","P_i","C_j"]
assignment_DF = pd.DataFrame(columns=assignment_column)

In [36]:
def seeding_A(instance):
    clusters = instance.NumberOfClusters.value
    points = instance.NumberOfPoints.value
    for x in range(1,points+1):
        instance.Assignment[x,((x-1)%clusters)+1]=1

In [41]:
def seeding_B(instance, df):
    clusters = instance.NumberOfClusters.value
    sliced_df = df.sample(clusters)
    ## print(sliced_df.values)
    for row in sliced_df.values:
        for centroid_id in range(clusters):
            for dim in range(dimensions):
                instance.Centroid[centroid_id+1,dim+1]=row[dim]

In [39]:
df = pd.read_csv("../Data/Mushroom/mushroom_data.csv")
df.head()

,cap-shape_bell,cap-shape_conical,cap-shape_convex,cap-shape_flat,cap-shape_knobbed,cap-shape_sunken,cap-surface_fibrous,cap-surface_grooves,cap-surface_scaly,cap-surface_smooth,...,population_scattered,population_several,population_solitary,habitation_grasses,habitation_leaves,habitation_meadows,habitation_paths,habitation_urban,habitation_waste,habitation_woods
0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [42]:
seeding_B(original_instance2, df=df)
count=0

[[0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0
  1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 1 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1
  0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
  1 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1
  0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 1 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1
  0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0
  0 0 1 0 0 0 0 0 0]]


In [44]:
past_instance2 = None

In [45]:
while(True):
    median_columns = ["group"]+["x_"+str(d) for d in range(1,dimensions+1)]
    median_df = pd.DataFrame(columns =median_columns)
    
    Soln2 = Opt.solve(current_instance2)
    current_instance2.solutions.load_from(Soln2)
    
    print("Current_instance 2: ", count)
    print("Termination Condition was "+str(Soln2.Solver.Termination_condition))
    
    ##display(current_instance2)
    ## Assignment Record
    for i in range(1,points+1):
        for j in range(1,clusters+1):
            if(current_instance2.Assignment[i,j] == 1):
                arr = [count, i, j]
                assignment_DF.loc[len(assignment_DF)] = arr
                arr2 = [j]
                for d  in range(1,dimensions+1):
                    arr2.append(value(current_instance2.Point[i,d]))
                median_df.loc[len(median_df)] = arr2
    
    ## Centroid Location
    for j in range(1,clusters+1):
        arr = [count, j]
        for d  in range(1,dimensions+1):
            arr.append(value(current_instance2.Centroid[j,d]))
        centroid_DF.loc[len(centroid_DF)] = arr
        
    print(median_df.head())
    print(median_df.groupby("group").median())
    print(median_df.groupby("group").median().values)
    median_val= median_df.groupby("group").median().values
    
    for row in median_val:
        for centroid_id in range(clusters):
            for dim in range(dimensions):
                original_instance2.Centroid[centroid_id+1,dim+1]=row[dim]
    
    if(past_instance2 is not None):
        if np.array_equal(median_val, past_instance2):
            break
    past_instance2 = median_val
    count+=1
    print()

NameError: name 'current_instance2' is not defined

In [ ]:
print("Time to complete:", time.time() - start_time)

In [42]:
assignment_DF.to_csv(folder+dat_location+seeding+"_Assignment_INF_NORM.csv")

In [43]:
centroid_DF.to_csv(folder+dat_location+seeding+"_Centroid_INF_NORM.csv")

In [ ]:
count